In [ ]:
# @author: Ishman Mann
# @date: 13/10/2022
# 
# @description:
#   Classification model for CIFAR-10 dataset using a CNN in pyTorch
#
# @resources:
#
#
# @notes:
#   Only push to 'ishman-pytorch' branch!!
#   Use conda env instead of a virtualenv file 
#
# @todo:
#   Build model
#   Train model -- think of where to put softmax, crossEntropyLoss has softmax in it, so I can't 
#                  put softmax in my model layers. When computing accuracy, just call softmax there 
#   Evaluate model, use torchmetrics
#   Create a confusion matrix
#   Add image augmentation
#   Add model saving
#   Hyperparameter tuning

In [ ]:
######################################################################################################
# Imports

import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd

import shutil
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Set device as gpu
device = "cuda" if torch.cuda.is_available() else "cpu"

# Set random seeds
torch.manual_seed(42)

'd:\\Documents\\computer-vision-bootcamp-pyTorch'

In [ ]:
# Delete and recreate datasets folder (For Google Colab only)
if os.path.exists("./datasets"):
  shutil.rmtree("./datasets", ignore_errors=True)
  os.makedirs("./datasets")

In [ ]:
######################################################################################################
# Loading testing and training data

trainData = datasets.CIFAR10(
    root="./datasets",
    train=True, # get train data
    download=True,
    transform=ToTensor(), # converts PIL to torch.tensor
    target_transform=None #dont transform targets (labels)!
)

testData = datasets.CIFAR10(
    root="./datasets",
    train=False, # get test data
    download=True,
    transform=ToTensor(),
    target_transform=None #dont transform targets (labels)!
) 

CLASS_NAMES = trainData.classes

In [ ]:
# Split off some validation data
TRAIN_LENGTH = int(len(trainData.data)*0.8)
VALIDATE_LENGTH = int(len(trainData.data)*0.2)
trainData, validateData = torch.utils.data.random_split(trainData, [TRAIN_LENGTH, VALIDATE_LENGTH])

In [ ]:
# Viewing a sample image
image, label = trainData[0]
imagePermuted = image.permute(1,2,0)
print(imagePermuted.shape)
plt.imshow(imagePermuted)
plt.title(CLASS_NAMES[label])

In [ ]:
# Batch the data using DataLoader

BATCH_SIZE = 32

trainDataloader = DataLoader(trainData, batch_size=BATCH_SIZE, shuffle=True)
validateDataloader = DataLoader(validateData, batch_size=BATCH_SIZE, shuffle=False)
testDataloader = DataLoader(testData, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
######################################################################################################
# Create the model

class CIFAR10ModelV1(nn.Module):

  def __init__(self, inputChannels: int, hiddenUnits: int, outputShape: int):
    super().__init__()

    # Convolution & pooling layers
    self.cnn_layer_1 = nn.sequential(
      nn.Conv2d(in_channels=inputChannels, out_channels=hiddenUnits,
                kernel_size=3, stride=1, padding="same"),
      nn.Relu(),
      nn.BatchNorm2d(num_features=hiddenUnits, eps=1e-05, momentum=0.1, affine=True),
      nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                kernel_size=3, stride=1, padding="same"),
      nn.Relu(),
      nn.BatchNorm2d(num_features=hiddenUnits, eps=1e-05, momentum=0.1, affine=True),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.cnn_layer_2 = nn.sequential(
      nn.Conv2d(in_channels=hiddenUnits, out_channels=2*hiddenUnits,
                kernel_size=3, stride=1, padding="same"),
      nn.Relu(),
      nn.BatchNorm2d(num_features=2*hiddenUnits, eps=1e-05, momentum=0.1, affine=True),
      nn.Conv2d(in_channels=2*hiddenUnits, out_channels=2*hiddenUnits,
                kernel_size=3, stride=1, padding="same"),
      nn.Relu(),
      nn.BatchNorm2d(num_features=2*hiddenUnits, eps=1e-05, momentum=0.1, affine=True),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )

    # Fully connected layers
    self.classifier = nn.sequential(
        nn.Flatten(),
        nn.Linear(in_features=2*hiddenUnits*8*8, # 8*8 comes from maxpooling 32*32 pixels twice
                   out_features=outputShape)
    )

    # Placing model layers in forward()
    def forward(self, x: torch.Tensor):
      x = self.cnn_layer_1(x)
      x = self.cnn_layer_2(x)
      x = self.classifier(x)
